In [1]:
using CSV
using Glob
using DataFrames
using Missings
using ProgressBars
using ArchGDAL

┌ Info: Precompiling ArchGDAL [c9ce4bd3-c3d5-55b8-8973-c0e20141b8c3]
└ @ Base loading.jl:1342


In [2]:
sensor_meta = CSV.read("../data/good_sensors.csv", DataFrame)

,ID,fwy_stable,dir_stable,lanes_stable,max_shift_meters,Latitude,Longitude
,Int64,Bool,Bool,Bool,Float64,Float64,Float64
1,311831,1,1,1,59.5162,38.4098,-121.484
2,311844,1,1,1,80.6189,38.4128,-121.484
3,311845,1,1,1,0.0,38.4062,-121.483
4,311847,1,1,1,91.9547,38.4283,-121.488
5,312010,1,1,1,9.62967,38.5638,-121.493
6,312098,1,1,1,85.7125,38.559,-121.476
7,312103,1,1,1,12.8604,38.561,-121.463
8,312132,1,1,1,9.56544,38.4098,-121.484
9,312133,1,1,1,53.2263,38.4282,-121.488


In [4]:
uzas = DataFrame(ArchGDAL.getlayer(ArchGDAL.read("../data/tl_2020_us_uac10.shp"), 0))
nothing

In [5]:
sensor_meta.geom = passmissing(ArchGDAL.createpoint).(sensor_meta.Longitude, sensor_meta.Latitude)
nothing

In [12]:
names(uzas)

13-element Vector{String}:
 ""
 "UACE10"
 "GEOID10"
 "NAME10"
 "NAMELSAD10"
 "LSAD10"
 "MTFCC10"
 "UATYP10"
 "FUNCSTAT10"
 "ALAND10"
 "AWATER10"
 "INTPTLAT10"
 "INTPTLON10"

In [16]:
# filter to just California, and just urbanized areas rather than urban clusters
ca_uza_geom = uzas[occursin.(["CA"], uzas.NAME10) .& (uzas.UATYP10 .== "U"), ""]
nothing

In [17]:
@assert !any(ismissing.(sensor_meta.geom))

In [18]:
sensor_meta.urban = map(sensor_meta.geom) do g
    for poly in ca_uza_geom
        if ArchGDAL.contains(poly, g)
            return true
        end
    end    
    return false  # not in any UZA
end
nothing

In [19]:
sum(sensor_meta.urban)

16043

In [20]:
sum(.!sensor_meta.urban)

2791

In [24]:
@assert !any(ismissing.(sensor_meta.urban))

In [21]:
CSV.write("../data/sensor_meta_geo.csv", sensor_meta)

"../data/sensor_meta_geo.csv"

false